In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

import torch
import torch.nn as nn
from torch.functional import F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.data import Data, HeteroData
from torch_geometric.loader import DataListLoader, DataLoader
from torch_geometric.utils.convert import to_networkx
import networkx as nx

from sklearn.preprocessing import MinMaxScaler

/var/folders/0t/9z26630n32l3qw5w80zvvjqw0000gn/T/ipykernel_14728/1010190927.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Load network data: node features, edge_features and edge_index
node_features = pd.read_csv('network/node_features.csv', index_col=0, header=0)
node_features = node_features.astype(np.float64).fillna(value=0)
branch_index = pd.read_csv('network/branch_index.csv', index_col=0, header=0)
trafo_index = pd.read_csv('network/trafo_index.csv', index_col=0, header=0)
branch_attr = pd.read_csv('network/branch_attr.csv', index_col=0, header=0)
trafo_attr = pd.read_csv('network/trafo_attr.csv', index_col=0, header=0)

# Display the shapes
print(node_features.shape, branch_index.shape, trafo_index.shape, branch_attr.shape, trafo_attr.shape)




(1354, 8) (1751, 2) (240, 2) (1751, 3) (240, 5)


In [3]:
# Convert indices to torch tensors
branch_index = torch.tensor(branch_index.to_numpy().T, dtype=torch.long)
trafo_index = torch.tensor(trafo_index.to_numpy().T, dtype=torch.long)


In [4]:
# Load synthetic 'p_mw' at load buses
p_load_data = pd.read_csv('synthetic_data/p_load_data.csv', index_col=0, header=0)
p_load_data.shape


(621, 1000)

In [5]:
# Load synthetic 'p_mw' at generator buses
p_gen_data = pd.read_csv('synthetic_data/p_gen_data.csv', index_col=0, header=0)
p_gen_data.shape

(259, 1000)

In [6]:
# Load scaled 'p_mw' at all buses
p_bus_data = pd.read_csv('synthetic_data/p_bus_data.csv', index_col=0, header=0)
p_bus_data.shape

(1354, 1000)

In [7]:
p_slack_max_data = pd.read_csv('synthetic_data/p_slack_max_data.csv', index_col=0, header=0)
p_slack_max_data.shape

(1000, 1)

In [8]:
## All bus
bus = np.arange(node_features.shape[0])

## All gen bus
gen_bus = pd.read_csv('zones/re_gen_bus.csv', 
                      index_col=None, 
                      header=None).to_numpy()
gen_bus = gen_bus.reshape((-1)) - 1
# Gen bus mask
gen_no = np.isin(bus, gen_bus)

# All load bus
load_bus = pd.read_csv('zones/load_bus.csv', 
                       index_col=None, 
                       header=None).to_numpy()
load_bus = load_bus.reshape((-1)) - 1
# Load bus mask
load_no = np.isin(bus, load_bus)

# Wind bus
re_gen_bus = pd.read_csv('zones/wind_bus.csv', 
                         index_col=None, 
                         header=None).to_numpy()
re_gen_bus = re_gen_bus.reshape((-1)) - 1
# Wind bus mask
re_gen_no = np.isin(gen_bus, re_gen_bus)
not_re_gen_no = ~re_gen_no

In [9]:
re_gen_bus.shape, re_gen_no.shape, not_re_gen_no.shape, gen_no.shape, load_no.shape

((207,), (259,), (259,), (1354,), (1354,))

In [12]:

# Define bus arrays
bus = np.arange(node_features.shape[0])

# Load and generation buses
load_bus = pd.read_csv('zones/load_bus.csv', index_col=None, header=None).to_numpy().reshape((-1)) - 1
gen_bus = pd.read_csv('zones/re_gen_bus.csv', index_col=None, header=None).to_numpy().reshape((-1)) - 1
re_gen_bus = pd.read_csv('zones/wind_bus.csv', index_col=None, header=None).to_numpy().reshape((-1)) - 1

# Ensure load_bus and re_gen_bus are valid indices in node_features
valid_load_indices = [idx for idx in load_bus if idx in node_features.index]
valid_re_gen_indices = [idx for idx in re_gen_bus if idx in node_features.index]

# Verify shapes of masks and data
print(f"Total number of buses: {len(bus)}")
print(f"Number of valid load buses: {len(valid_load_indices)}")
print(f"Number of valid wind generation buses: {len(valid_re_gen_indices)}")
print(f"Shape of p_load_data: {p_load_data.shape}")
print(f"Shape of p_gen_data: {p_gen_data.shape}")

# Store multiple Data() in a list
data_list = []

for i in range(p_load_data.shape[1]):
    # Ensure we use only the relevant portion of p_load_data
    load_values = p_load_data.iloc[:len(valid_load_indices), i].values
    if len(valid_load_indices) != len(load_values):
        raise ValueError(f"Mismatch in lengths: valid_load_indices={len(valid_load_indices)}, load_values={len(load_values)}")
    node_features.loc[valid_load_indices, 'load_p_mw'] = load_values

    # Ensure the correct length of values for re_gen_no mask
    re_gen_values = p_gen_data.iloc[:len(valid_re_gen_indices), i].values
    if len(re_gen_values) != len(valid_re_gen_indices):
        raise ValueError(f"Mismatch in lengths: re_gen_values={len(re_gen_values)}, valid_re_gen_indices={len(valid_re_gen_indices)}")
    node_features.loc[valid_re_gen_indices, 'max_gen_p_mw'] = re_gen_values
    node_features.loc[valid_re_gen_indices, 'min_gen_p_mw'] = re_gen_values

    # Set max_p_mw at the slack bus
    node_features.loc[271, 'max_gen_p_mw'] = p_slack_max_data.iloc[i].values

    # Convert features and targets to tensors
    X = torch.from_numpy(node_features.to_numpy()).float()
    y = torch.from_numpy(p_bus_data.iloc[:, i].to_numpy().reshape((-1))).float()

    # Create HeteroData object
    data = HeteroData()
    data['node'].x = X
    data['node'].y = y

    # Set edge index and attributes
    data['node', 'branch', 'node'].edge_index = branch_index
    data['node', 'branch', 'node'].edge_attr = torch.from_numpy(branch_attr.to_numpy()).float()
    data['node', 'trafo', 'node'].edge_index = trafo_index
    data['node', 'trafo', 'node'].edge_attr = torch.from_numpy(trafo_attr.to_numpy()).float()

    data_list.append(data)

Total number of buses: 1354
Number of valid load buses: 91
Number of valid wind generation buses: 23
Shape of p_load_data: (621, 1000)
Shape of p_gen_data: (259, 1000)


In [14]:
## Save train and test dataset
train_size = int(0.7*len(data_list))
val_size = int(0.1*len(data_list))
test_size = int(0.2*len(data_list))

# Create PyTorch Geometric DataLoader()
batch_size = 1
train_data_loader = DataLoader(dataset=data_list[:train_size], batch_size=batch_size, shuffle=False)
val_data_loader = DataLoader(dataset=data_list[train_size:train_size+val_size], batch_size=batch_size, shuffle=False)
test_data_loader = DataLoader(dataset=data_list[-test_size:], batch_size=batch_size, shuffle=False)

# Save train and test datasets
torch.save(train_data_loader, 'train_test_dataset/node_prediction_train_dataset.pt')
torch.save(val_data_loader, 'train_test_dataset/node_prediction_val_dataset.pt')
torch.save(test_data_loader, 'train_test_dataset/node_prediction_test_dataset.pt')